# APF

In [ ]:
%matplotlib inline

## APF test

In [ ]:
# === PARAMETERS ===
NUM_AGENTS = 500
GOAL_TOL = 0.001
STEP_SIZE = 0.05
K_ATTR = 1.0      # Attractive potential gain
K_REP = 0.0002    # Repulsive potential gain
REPULSION_RADIUS = 0.55
MAX_REP_FORCE = 1.0
MIN_DIST = 0.05   # Prevent division by very small values
MAX_STEP = 0.07   # Clamp maximum step per agent
DAMPING = 0.7     # Optional: smooth movement

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# === INITIAL CONDITIONS ===
np.random.seed(0)
positions = np.random.rand(NUM_AGENTS, 2) * np.array((-2, 1))

# === RANDOM GAUSSIAN GOALS ===
# goals = np.tile(np.array([1, 0]), (NUM_AGENTS, 1) )
goal_mean = np.array([1.0, 1.0])
goal_cov = np.array([[0.05, 0], [0, 1]])  # Small spread
goals = np.random.multivariate_normal(mean=goal_mean, cov=goal_cov, size=NUM_AGENTS)

# === APF PLANNING FUNCTION ===
def compute_apf_forces(pos, goals):
    forces = np.zeros_like(pos)
    for i in range(len(pos)):
        # Attractive force
        diff = goals[i] - pos[i]
        f_attr = K_ATTR * diff

        # Repulsive force from other agents
        f_rep = np.zeros(2)
        
        for j in range(len(pos)):
            if i == j:
                continue
            diff_ij = pos[i] - pos[j]
            dist = np.linalg.norm(diff_ij)
            if dist < REPULSION_RADIUS and dist > 1e-6:
                # Clamp small distances to avoid singularity
                dist_clamped = max(dist, MIN_DIST)
                rep_mag = K_REP * (1.0 / dist_clamped - 1.0 / REPULSION_RADIUS) / (dist_clamped**2)
                rep_mag = np.clip(rep_mag, 0, MAX_REP_FORCE)
                f_rep += rep_mag * (diff_ij / dist_clamped)

        forces[i] = f_attr + f_rep
    return forces

# === VISUALIZATION SETUP ===
fig, ax = plt.subplots()
scat = ax.scatter([], [], c='blue')
goal_plot = ax.scatter(goals[:, 0], goals[:, 1], c='green', marker='x')
ax.set_xlim(-1, 3)
ax.set_ylim(-1, 3)
ax.set_aspect('equal')

# === SIMULATION LOOP ===
stopped = False

def update(frame):
    global positions, stopped

    if not stopped:
        dists_to_goals = np.linalg.norm(positions - goals, axis=1)
        if np.any(dists_to_goals < GOAL_TOL):
            stopped = True
        else:
            forces = compute_apf_forces(positions, goals)
            for i in range(len(positions)):
                step = STEP_SIZE * forces[i]
                norm = np.linalg.norm(step)
                if norm > MAX_STEP:
                    step = MAX_STEP * step / norm
                positions[i] += DAMPING * step  # Apply damping

    scat.set_offsets(positions)
    return scat,

ani = animation.FuncAnimation(fig, update, frames=200, interval=100, blit=True)
plt.title("Multi-Agent APF with Stability Control")
ani.save("solutions/apf_stability.mp4", writer="ffmpeg", fps=24)
plt.close()

from IPython.display import Video
Video(filename="solutions/apf_stability.mp4", embed=True)

## APF waypoint

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import linear_sum_assignment
from scipy.spatial.distance import cdist

from st_gaussian_prm.solvers.micro.apf import APF

num_agents = 30

# generate per agent trajectory

means = np.array([(0., 0.), (5., 0.), (8, 1), (10, 5)])
covs = np.array([[(1, 0), (0, 5)], [(3, 0), (0, 1)], [(1, 0), (0, 1)],[(1, 0), (0, 1)]])

class GaussianNode:
    def __init__ (self, mean, cov):
        self.mean = mean
        self.cov = cov
    
g_nodes = []

for mean, cov in zip(means, covs):
    g_nodes.append(GaussianNode(mean, cov))

g_paths = np.tile(np.arange(4), [num_agents, 1])

# means = np.array([(0., 0.), (5., 0.)])
# covs = [np.eye(2) for _ in means]


apf_config = {
    "k_attr": 0.5,
    "k_rep": 5.2,
    "step_size": 0.05,
    "repulsion_radius": 0.4,
    "goal_chisq_threshold": 5.991, # 95% CI
    "max_rep_force": 1.0,
    "min_dist": 0.05,
    "max_step": 0.07,
    "damping": 0.7
}

trajectories =APF(g_paths, g_nodes, **apf_config).solve()

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np


fig, ax = plt.subplots()
colors = plt.cm.jet(np.linspace(0, 1, num_agents))
points = [ax.plot([], [], 'o', color=colors[i])[0] for i in range(num_agents)]

ax.plot(trajectories[:, -1, 0], trajectories[:, -1, 1], 'x', color="green") # plot goals
ax.set_xlim(np.min(trajectories[:, :, 0]) - 1, np.max(trajectories[:, :, 0]) + 1)
ax.set_ylim(np.min(trajectories[:, :, 1]) - 1, np.max(trajectories[:, :, 1]) + 1)
ax.set_title("APF Trajectories")
ax.set_aspect("equal")

SKIP = 10
def update(frame):
    for i in range(num_agents):
        x, y = trajectories[i, frame]
        points[i].set_data([x], [y])
    return points 

ani = animation.FuncAnimation(fig, update, frames=range(0, trajectories.shape[1], SKIP), interval=50, blit=True)

ani.save("solutions/apf.mp4", writer="ffmpeg", fps=24)
plt.close()


from IPython.display import Video
Video(filename="solutions/apf.mp4", embed=True)
